In [1]:
import datetime, time
import simpy

import shapely.geometry
from simplekml import Kml, Style

# package(s) for data handling
import pandas as pd
import numpy as np

import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot
import openclsim.plugins as plugin

In [2]:
#simulation_start = pd.Timestamp().to_datetime64()
simulation_start  = datetime.datetime(2010,1,1)

my_env = simpy.Environment(initial_time=simulation_start.timestamp())

registry = {}

# The generic site class
Site = type(
    "Site",
    (
        core.Identifiable,  # Give it a name
        core.Log,  # Allow logging of all discrete events
        core.Locatable,  # Add coordinates to extract distance information and visualize
        core.HasContainer,  # Add information on the material available at the site
        core.HasResource,
    ),  # Add information on serving equipment
    {},
)  # The dictionary is empty because the site type is generic

# The generic class for an object that can move and transport (a TSHD for example)
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,  # Give it a name
        core.Log,  # Allow logging of all discrete events
        core.ContainerDependentMovable,  # A moving container, so capacity and location
        core.Processor,  # Allow for loading and unloading
        core.HasResource,  # Add information on serving equipment
        core.HasCosts,  # Add information on costs
        core.LoadingFunction,  # Add a loading function
        core.UnloadingFunction,  # Add an unloading function
    ),
    {},
)

TestMoveActivity =  type(
    "TestMoveActivity",
    (
        plugin.HasTestPluginMoveActivity,
        plugin.HasWeatherPluginMoveActivity,
        model.MoveActivity,  # the order is critical!
    ),
    {},
)

In [3]:
# Information on the extraction site - the "from site" - the "win locatie"
location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)  # lon, lat

data_from_site = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Winlocatie",  # The name of the site
    "ID": "6dbbbdf4-4589-11e9-a501-b469212bff5b",  # For logging purposes
    "geometry": location_from_site,  # The coordinates of the project site
    "capacity": 10,  # The capacity of the site
    "level": 2,
}  # The actual volume of the site

# Information on the dumping site - the "to site" - the "dump locatie"
location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)  # lon, lat

data_to_site = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Dumplocatie",  # The name of the site
    "ID": "6dbbbdf5-4589-11e9-82b2-b469212bff5b",  # For logging purposes
    "geometry": location_to_site,  # The coordinates of the project site
    "capacity": 10,  # The capacity of the site
    "level": 0,
}  # The actual volume of the site (empty of course)

from_site = Site(**data_from_site)
to_site = Site(**data_to_site)

init
level: 2
completed init
init
level: 0
completed init


In [4]:
data_hopper = {
    "env": my_env,  # The simpy environment
    "name": "Hopper 01",  # Name
    "ID": "6dbbbdf6-4589-11e9-95a2-b469212bff5b",  # For logging purposes
    "geometry": location_from_site,  # It starts at the "from site"
    "loading_rate": 1,  # Loading rate
    "unloading_rate": 1,  # Unloading rate
    "capacity": 5,  # Capacity of the hopper - "Beunvolume"
    "compute_v": lambda x: 10,
    "weekrate": 7,
}


hopper = TransportProcessingResource(**data_hopper)

init
level: 0
completed init


In [5]:
metocean_df = pd.read_csv("../demo/unit_test_weather.csv")
metocean_df = metocean_df.set_index(pd.to_datetime(metocean_df["Time"], dayfirst=True))
metocean_df = metocean_df.sort_index()

criteria_data = {
    "event_name":"this",
    "condition":"Hs [m]",
    "maximum":3,
    "window_length":datetime.timedelta(hours=1)
}

crit = core.WorkabilityCriterion(**criteria_data)

move_activity_data = {
    "env": my_env,
    "name": "Soil movement",
    "registry": registry,
    "mover": hopper,
    "destination": to_site,
    "metocean_criteria":[crit],
    "metocean_df": metocean_df,
}

activity = TestMoveActivity(**move_activity_data)

my_env.run()

start event instance None
check weather plugin
True
True
True
regrister weather plugin
Mover_move before mover resource request
put_callback - id_ default
{}
put_callback - id_ default
{}
put_callback - id_ default
{}
Mover_move after mover resource request
weatherPlugin start preprocess
[['2010-01-01T04:20:00.000000000' '2010-01-01T15:30:00.000000000']
 ['2010-01-03T04:20:00.000000000' '2010-01-04T21:30:00.000000000']
 ['2010-01-05T04:20:00.000000000' '2010-01-07T09:30:00.000000000']
 ['2010-01-08T04:20:00.000000000' '2010-01-08T15:30:00.000000000']]
[15600.0]
we have to wait for 15600.0
delay processing 15600.0
before delay 1262300400.0
after delay 1262316000.0
plugin_data mover: <__main__.TransportProcessingResource object at 0x0000017212EAED88>
plugin_data origin: <__main__.TransportProcessingResource object at 0x0000017212EAED88>
plugin_data destination: <__main__.Site object at 0x0000017212EAEF08>
plugin_data engine_oder: 1.0
plugin_data activity_log: <__main__.TestMoveActivity o

In [7]:
activity.log
log_df = pd.DataFrame(activity.log)
data = log_df[["Message", "ActivityState", "Timestamp", "Value", "ActivityID"]]

data

,Message,ActivityState,Timestamp,Value,ActivityID
0,move activity Soil movement of Hopper 01 to Du...,UNKNOWN,2010-01-01 00:00:00.000000,-1,6dbbbdf7-4589-11e9-bf3b-b469212bff5b
1,move activity Soil movement of Hopper 01 to Du...,WAIT_START,2010-01-01 00:00:00.000000,-1,6dbbbdf7-4589-11e9-bf3b-b469212bff5b
2,move activity Soil movement of Hopper 01 to Du...,WAIT_STOP,2010-01-01 04:20:00.000000,-1,6dbbbdf7-4589-11e9-bf3b-b469212bff5b
3,move activity pre-procesisng test plugin,UNKNOWN,2010-01-01 04:20:00.000000,-1,6dbbbdf7-4589-11e9-bf3b-b469212bff5b
4,move activity Soil movement of Hopper 01 to Du...,START,2010-01-01 04:20:00.000000,-1,6dbbbdf7-4589-11e9-bf3b-b469212bff5b
5,move activity post-procesisng test plugin,UNKNOWN,2010-01-01 04:35:42.824591,-1,6dbbbdf7-4589-11e9-bf3b-b469212bff5b
6,move activity Soil movement of Hopper 01 to Du...,STOP,2010-01-01 04:35:42.824591,-1,6dbbbdf7-4589-11e9-bf3b-b469212bff5b
